In [ ]:
!nvidia-smi

In [ ]:
#!unzip /content/dataset.zip

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
imbalance_data = pd.read_csv("imbalanced_data.csv")

In [ ]:
imbalance_data.head()

In [ ]:
imbalance_data.columns

## EDA

In [ ]:
sns.countplot(x = 'label', data = imbalance_data)

In [ ]:
imbalance_data["label"].value_counts()

##
- 0 = No Hate
- 1 = Hate

In [ ]:
imbalance_data.shape

In [ ]:
imbalance_data.isnull().sum()

In [ ]:
imbalance_data.drop("id", axis= 1, inplace= True)

In [ ]:
imbalance_data.head()

In [ ]:
raw_data = pd.read_csv("raw_data.csv")
raw_data.head()

In [ ]:
raw_data.shape

In [ ]:
raw_data.isnull().sum()

In [ ]:
raw_data.columns

In [ ]:
raw_data.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'],axis = 1, inplace=True)

In [ ]:
raw_data.head()

In [ ]:
raw_data["class"].value_counts()

In [ ]:
raw_data["class"].unique()

In [ ]:
sns.countplot(x = 'class', data = raw_data)

- class 0: hate
- class 1: abusive
- class 2: no hate

In [ ]:
# Combine class 1 and 0
raw_data[raw_data["class"]==0]["class"] = 1

In [ ]:
raw_data.head()

In [ ]:
raw_data["class"].unique

In [ ]:
raw_data['class'].value_counts()

In [ ]:
raw_data[raw_data["class"]==0]

In [ ]:
# Replace the value of 0 to 1
raw_data['class'].replace({0:1},inplace=True)

In [ ]:
raw_data.head()

In [ ]:
raw_data['class'].unique()

In [ ]:
sns.countplot(x = 'class', data = raw_data)

In [ ]:
raw_data["class"].value_counts()

In [ ]:
# let's replace the values of 2 to 0
raw_data['class'].replace({2:0},inplace=True)

In [ ]:
raw_data.head()

In [ ]:
raw_data['class'].unique()

In [ ]:
sns.countplot(x='class', data=raw_data)

In [ ]:
imbalance_data.head()

In [ ]:
raw_data.head()

In [ ]:
# Let's change the name of the 'class' to 'label'
raw_data.rename(columns={'class':'label'},inplace=True)

In [ ]:
raw_data.head()

In [ ]:
imbalance_data.head()

In [ ]:
# merge
frame = [imbalance_data,raw_data]
df = pd.concat(frame)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
sns.countplot(x = 'label', data = df)

In [ ]:
df['label'].value_counts()

# Pre-processing step

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
#nltk.download('all')

In [ ]:
# Let's apply stemming and stopwords on the data
stemmer = nltk.SnowballStemmer("english")
stopword = set(stopwords.words('english'))

In [ ]:
stopword

In [ ]:
def data_cleaning(words):
    words = str(words).lower()
    words = re.sub('\[.*?\]', '', words)
    words = re.sub('https?://\S+|www\.\S+', '', words)
    words = re.sub('<.*?>+', '', words)
    words = re.sub('[%s]' % re.escape(string.punctuation), '', words)
    words = re.sub('\n', '', words)
    words = re.sub('\w*\d\w*', '', words)

    # correct stopword removal
    words = [word for word in words.split(' ') if word not in stopword]
    words = " ".join(words)

    # correct stemming
    words = [stemmer.stem(word) for word in words.split(' ')]
    words = " ".join(words)

    return words


In [ ]:
df['tweet'][1]

In [ ]:
df['tweet'] = df['tweet'].apply(data_cleaning)

In [ ]:
df['tweet'][1]

In [ ]:
x = df['tweet']
y = df['label']

In [ ]:
from sklearn.model_selection import train_test_split

# Let's split the data into train and test

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)


In [ ]:
print(len(x_train),len(y_train))
print(len(x_test),len(y_test))

In [ ]:
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


In [ ]:
print(len(x_train),len(y_train))
print(len(x_test),len(y_test))

In [ ]:
import numpy as np

y_train = np.array(y_train)
y_test = np.array(y_test)


## Feature Engineering

In [ ]:
#!pip install keras
#!pip install tensorflow

In [ ]:
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences


In [ ]:
max_words = 50000
max_len = 300

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

sequences = tokenizer.texts_to_sequences(x_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [ ]:
sequences_matrix

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 100, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.build(input_shape=(None, max_len))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
# starting model training
history = model.fit(sequences_matrix,y_train,batch_size=128,epochs = 5,validation_split=0.2)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
test_sequences_matrix

In [ ]:
# Model evaluation
accr = model.evaluate(test_sequences_matrix,y_test)

In [ ]:
lstm_prediction = model.predict(test_sequences_matrix)

In [ ]:
res = []
for prediction in lstm_prediction:
    if prediction[0] < 0.5:
        res.append(0)
    else:
        res.append(1)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print(confusion_matrix(y_test,res))

In [ ]:
print(accuracy_score(y_test,res))

In [ ]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Let's save the mdoel.
model.save("model.h5")

In [ ]:
load_model=keras.models.load_model("model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [ ]:
# Let's test our model on custom data.
test = 'i love this movie'

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    words = text.split(' ')
    words = [word for word in words if word not in stopword]
    words = [stemmer.stem(word) for word in words]
    text = " ".join(words)
    return text

test=[clean_text(test)]
print(test)

seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)
print(seq)

pred = load_model.predict(padded)

print("pred", pred)
if pred<0.5:
    print("no hate")
else:
    print("hate and abusive")
